<img src="fcn_model.png">

Reference 1 : https://github.com/shekkizh/FCN.tensorflow/blob/master/FCN.py
<br>Reference 2 : https://modulabs-biomedical.github.io/FCN
<br>Reference 3 : https://stackoverflow.com/questions/38160940/how-to-count-total-number-of-trainable-parameters-in-a-tensorflow-model

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim

C:\Users\user\Anaconda3\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# image size

height = 224
width = 224
num_of_channels = 3
num_of_classes = 21

keep_prob = 0.5

In [3]:
image = tf.placeholder(tf.float32, [None, height, width, num_of_channels])

In [4]:
def fcn8(image, num_of_classes, keep_prob):
    
    "Feature Extraction"
    # output shape = [None, H/2, H/2, 96]
    conv1 = slim.conv2d(image, 96, [3, 3], scope = 'conv1_1')
        # (3*3*3 + 1) * 96 = 2,688 (Number of Weights)
    conv1 = slim.conv2d(conv1, 96, [3, 3], scope = 'conv1_2')
        # (3*3*96 + 1) * 96 = 83,040
    pool1 = slim.max_pool2d(conv1, [2, 2], scope = 'pool1', padding = 'SAME') 
    
    # output shape = [None, H/4, H/4, 256]
    conv2 = slim.repeat(pool1, 2, slim.conv2d, 256, [3, 3], scope = 'conv2')
        # (3*3*96 + 1) * 256 + (3*3*256 + 1) * 256 = 811,520
    pool2 = slim.max_pool2d(conv2, [2, 2], scope = 'pool2')
    
    # output shape = [None, H/8, H/8, 384]
    conv3 = slim.repeat(pool2, 3, slim.conv2d, 384, [3, 3], scope = 'conv3')
        # (3*3*256 + 1) * 384 + (3*3*384 + 1) * 384 * 2 = 3,540,096
    pool3 = slim.max_pool2d(conv3, [2, 2], scope = 'pool3', padding = 'SAME')
    
    # output shape = [None, H/16, H/16, 384]
    conv4 = slim.repeat(pool3, 3, slim.conv2d, 384, [3, 3], scope = 'conv4')
        # (3*3*384 + 1) * 384 * 3 = 3,982,464
    pool4 = slim.max_pool2d(conv4, [2, 2], scope = 'pool4', padding = 'SAME')
    
    # output shape = [None, H/32, H/32, 256]
    conv5 = slim.repeat(pool4, 3, slim.conv2d, 256, [3, 3], scope = 'conv5')
        # (3*3*384 + 1) * 256 + (3*3*256 + 1) * 256 * 2 = 2,065,152
    pool5 = slim.max_pool2d(conv5, [2, 2], scope = 'pool5', padding = 'SAME')
    
    
    "Feature-level Classificaiton"
    # output shape = [None, H/32, H/32, 4096]
    conv6 = slim.conv2d(pool5, 4096, [7, 7], scope = 'conv6')
        # (7*7*256 + 1) * 4,096 = 51,384,320
    conv6 = tf.nn.dropout(conv6, keep_prob = keep_prob)
    
    # output shape = [None, H/32, H/32, 4096]
    conv7 = slim.conv2d(conv6, 4096, [1, 1], scope = 'conv7')
        # (1*1*4,096 + 1) * 4,096 = 16,781,312
    conv7 = tf.nn.dropout(conv7, keep_prob = keep_prob)
    
    # output shape = [None, H/32, H/32, num_of_classes]
    conv8 = slim.conv2d(conv7, num_of_classes, [1, 1], scope = 'conv8')
        # (1*1*4,096 + 1) * 21(num_of_classes) = 86,037
    
    
    "Upsampling"
    conv_t1 = slim.conv2d_transpose(conv8, num_outputs = pool4.get_shape()[3], kernel_size = [4, 4], stride = 2)
        # (4*4*21 + 1) * 384 = 129,408
    fuse_1 = tf.add(conv_t1, pool4, name = 'fuse_1')
    
    conv_t2 = slim.conv2d_transpose(fuse_1, num_outputs = pool3.get_shape()[3], kernel_size = 4, stride = 2) 
        # (4*4*384 + 1) * 384 = 2,359,680
    fuse_2 = tf.add(conv_t2, pool3, name = 'fuse_2')
    
    conv_t3 = slim.conv2d_transpose(fuse_2, num_outputs = num_of_channels, kernel_size = 16, stride = 8)
        # (16*16*384 + 1) * 3 = 294,915
    
    
    "Segmentation"
    annotation_pred = tf.argmax(conv_t3, dimension = num_of_channels, name="prediction")
    
    return tf.expand_dims(annotation_pred, dim = num_of_channels)

In [5]:
fcn8 = fcn8(image, num_of_classes, keep_prob)

In [6]:
print("The Output Shape : " + str(fcn8.shape))

The Output Shape : (?, 224, 224, 1)


In [11]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    total_parameters += variable_parameters
print("Number of Weights : " + format(total_parameters, ','))

Number of Weights : 81,520,632
